In [16]:
# ev centrality, deg centrality, cred score, toxicity score to be considered for users
import pandas as pd
import numpy as np
import glob

In [12]:
user_conspiracy = pd.read_csv("../files/tweets/result_cred_score_thresh_10.csv")
user_ev = pd.read_csv("../files/networks/user_ev_centrality.csv")
user_deg = pd.read_csv("../files/networks/user_deg_centrality.csv")
user_tox = pd.read_csv("../files/tweets/user_toxicity.csv")
suspended = pd.DataFrame()
for filename in glob.glob("../script_directory_output/suspended_users/output*.csv"):
    df = pd.read_csv(filename, lineterminator="\n", encoding="utf-8", low_memory=False)
    suspended = pd.concat([suspended, df], ignore_index=True)
suspended.reset_index(drop=True, inplace=True)
suspended.drop_duplicates(subset=["name"], inplace=True)
suspended
# botometer

,name,status
0,marzia38580873,"[{'code': 50, 'message': 'User not found.'}]"
1,Ivan__soli,ok
2,pat09_04,"[{'code': 50, 'message': 'User not found.'}]"
3,claudeger55,ok
4,ennioflyano,ok
...,...,...
839459,CollectifVrai,ok
839460,Mche10024327,ok
839461,albertotozzi3,"[{'code': 50, 'message': 'User not found.'}]"
839462,HanKer09062021,ok


In [13]:
tweets = user_conspiracy.merge(user_ev, left_on="user_screen_name", right_on="name", how="left")
tweets = tweets[tweets["name"].notna()]
tweets = tweets[["HIGH","LOW","UNKW","SOCIAL","conspiracy_score","name","ev"]]
tweets = tweets.merge(user_deg, on="name", how="left")
tweets = tweets.merge(user_tox, left_on="name", right_on="user_screen_name", how="left")
tweets = tweets.merge(suspended, on="name", how="left")
tweets

,HIGH,LOW,UNKW,SOCIAL,conspiracy_score,name,ev,deg_cen,text,toxicity,...,hashtags,created_at,in_reply_to_screen_name,rt_created_at,quoted_status_id,in_reply_to_user_id,date,count_status,toxicity_score,status
0,33,17267,171,0,1.0,infoitinterno,7.966932e-04,0.000324,"Iscrizioni scuola 2020, la guida di Bologna. D...",0.003662,...,[],Wed Jan 08 01:35:16 +0000 2020,NaN,NaN,NaN,NaN,2020-01-08,17189.0,0.031818,ok
1,0,5,0,13,1.0,Paolo90777,1.817897e-03,0.000139,Marco_Mandorli lucabattanta mat_brandi stefani...,0.059728,...,[],Fri Oct 02 19:23:54 +0000 2020,Marco_Mandorli,NaN,NaN,1.874501e+09,2020-10-02,46.0,0.101611,ok
2,0,7,3,1,1.0,ZiaGabriella1,5.881818e-03,0.000282,Se mi vaccino:,0.006768,...,[],Tue Dec 29 08:39:56 +0000 2020,NaN,Tue Dec 29 07:04:03 +0000 2020,NaN,NaN,2020-12-29,98.0,0.130837,ok
3,0,3,1,32,1.0,lapillo1,2.029138e-03,0.000111,Mascherine all'aperto? Inutili. Dati mortalità...,0.358705,...,[],Tue Oct 06 14:54:22 +0000 2020,NaN,Tue Oct 06 12:45:38 +0000 2020,NaN,NaN,2020-10-06,110.0,0.104499,ok
4,0,33,3,0,1.0,terry57,9.011427e-05,0.000019,"Lazio, il Tar accoglie il ricorso contro l’obb...",0.000768,...,[],Sat Oct 03 07:28:12 +0000 2020,NaN,NaN,NaN,NaN,2020-10-03,35.0,0.070365,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8472,0,0,10,0,NaN,CloCecchetto,2.454355e-05,0.000102,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ok
8473,0,0,10,0,NaN,castelPao,4.582850e-06,0.000005,"L'epidemiologo, vero e proprio esperto nell’or...",0.000154,...,[],Wed Jan 27 11:18:09 +0000 2021,NaN,NaN,NaN,NaN,2021-01-27,17.0,0.062314,ok
8474,0,0,10,0,NaN,traian_m,8.706800e-09,0.000014,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ok
8475,0,0,10,0,NaN,ComuneOristano,3.281069e-39,0.000005,Al via la campagna di vaccinazione antinfluenz...,0.044062,...,[],Tue Oct 20 14:14:52 +0000 2020,NaN,NaN,NaN,NaN,2020-10-20,11.0,0.018776,ok


In [22]:
print(np.mean(tweets[tweets["status"].str.contains("code")]["toxicity_score"]))
print(np.mean(tweets[tweets["status"].str.contains("ok")]["toxicity_score"]))
print(np.mean(tweets[tweets["status"].str.contains("code")]["conspiracy_score"]))
print(np.mean(tweets[tweets["status"].str.contains("ok")]["conspiracy_score"]))


0.12712205590623132
0.09259283733809207
0.3701219512195122
0.2764870317002882


In [21]:
# import numpy as np
# threshold_high_low = 20
# tweets = tweets[tweets['number_of_urls']>=threshold_high_low]
# tweets['conspiracy_score'] = np.around(((tweets['LOW'])/(tweets['HIGH']+tweets['LOW'])),2)
# tweets=tweets.sort_values(by=['conspiracy_score'], ascending=False)
# tweets.head(10)

In [14]:
tweets[["conspiracy_score","ev","deg_cen","toxicity_score"]].corr()

,conspiracy_score,ev,deg_cen,toxicity_score
conspiracy_score,1.000000,0.338556,0.024767,0.328490
ev,0.338556,1.000000,0.590345,0.276217
deg_cen,0.024767,0.590345,1.000000,0.061662
toxicity_score,0.328490,0.276217,0.061662,1.000000
